In [5]:
!pip install scipy
!pip install matplotlib

# Imports
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.gridspec as gridspec
import scipy
import scipy.io as io
import scipy.signal as signal
from scipy.signal import decimate 
from scipy.signal import cwt
import scipy.linalg as linalg
from scipy.stats import zscore
import random
import math

zsh:1: /Users/aimeejohnston/Desktop/beatLab/fnproject/fnve/bin/pip: bad interpreter: /Users/aimeejohnston/Desktop/beatLab/gammapotamus/fnproject/fnve/bin/python3.11: no such file or directory

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
zsh:1: /Users/aimeejohnston/Desktop/beatLab/fnproject/fnve/bin/pip: bad interpreter: /Users/aimeejohnston/Desktop/beatLab/gammapotamus/fnproject/fnve/bin/python3.11: no such file or directory

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [7]:
#Seiji's Preprocessing Functions
def labeling(x):
    """
    Input:
    x = 1 dimensional boolean array

    Output:
    x = 1 dimensional labeled array

    Description:
    This function takes a boolean array and applies unique numerical labeling by clusters of True elements.
    """
    # Initialize starting point
    label = 1

    # Apply unique ID to each cluster
    for idx, i in enumerate(x):
        if i == True:
            x[idx] = label
        elif i == False:
            if x[idx - 1] != False:
                label += 1

    # Return the new array
    return x

def compAnly(Track, fs):
    """
    Input:
    Track = Contains information on the session pertaining to the track
    fs = sampling frequency

    Output:
    lapID = Data stored in the format listed below

    lapID:
    Column 0: Trial Number
    Column 1: Port (1/2/3)
    Column 2: Correct (0/1) Note: This used to be (1/2), but I looked at the
    data and it was (0/1)
    Column 3: first approach/port/last departure/other (1/2/3/4)

    Description:
    This function extracts all the necessary metadata for the rat's position
    that will be used in future data processing.
    """

    from scipy.signal import decimate
    import numpy as np
    from statistics import median

    # Create the original matrix, calling known information
    lapID = np.array([ np.squeeze(Track["lapID"][0][0])[::int(1250/fs)] ])
    lapID = np.append(lapID, [ np.squeeze(Track["mazeSect"][0][0])[::int(1250/fs)] ], axis = 0)
    lapID = np.append(lapID, [ np.squeeze(Track["corrChoice"][0][0])[::int(1250/fs)] ], axis = 0)
    lapID = np.append(lapID, np.zeros((1, len(lapID[0]))), axis = 0)

    # Transpose to make lapID of appropriate dimensions
    lapID = lapID.T

    # Filter values and construct column 3
    lapID[np.in1d(lapID[:,1], np.array(range(4, 10)), invert = True), 1] = 0
    lapID[lapID[:, 1] > 6, 3] = 2
    lapID[lapID[:, 1] > 0, 1] = (lapID[lapID[:,1] > 0, 1] - 1) % 3 + 1
    lapID[lapID[:, 1] == 0, :] = 0
    # Identify samples corresponding to the first period when the animal enters the maze section
    for i in range(1, int(max(lapID[:, 0]) + 1)):
        r = labeling( 0 + np.logical_and((lapID[:, 0] == i), (lapID[:, 3] == 2)) )
        inds = np.where(lapID[:, 0] == i)[0]
        r1 = labeling( 0 + np.logical_and((lapID[:, 0] == i), (lapID[:, 3] == 0)) )
        lapID[r1 == max(np.multiply(r1, np.roll(r == 1,-1, axis = 0))), 3] = 1
        lapID[inds[inds > np.where(r > 0)[0][-1]], 3] = 3
        lapID[inds[lapID[inds, 3] == 0], 3] = 5
        lapID[r > 1, 3] = 4

    # Return structured data
    return lapID

def trainTestSplit(lapID, numTrain):
    """
    Input:
    lapID = lapID matrix
    numTrain = Some arbitrarily large number

    Output:
    trainInds = the index breakdown for the training dataset.

    Description:
    This is a specialized train/test split that generates a permutation (fold)
    for the number of prespecified cross validations. This function ensures
    that there is an even quantity of data for each output category within each
    cross validation fold.
    """

    import random
    import numpy as np

    # Initialize variable dimensions
    numCategories = max(lapID[:, 2])
    trainInds = np.zeros( (max(lapID[:, 1]), numTrain, numCategories) )
    minTrain = numTrain

    # Add a permutation of each output category for each cross validation fold
    for i in range(max(lapID[:, 1])):  # For each cross validation fold...
        for c in range(numCategories): # And for each port (output)...
            # Indices of values
            l = np.where(np.logical_and(lapID[:, 2] == c + 1, lapID[:, 1] != i + 1))[0]
            minTrain = min(minTrain, len(l))
            trainInds[i, 0:minTrain, c] = l[np.random.permutation(len(l))[0:minTrain]]

    # Reduce array to 2 dimensions in the form of (indices, CrossVal Folds)
    trainInds = trainInds[:, 0:minTrain, :]
    trainInds = np.concatenate((trainInds[:, :, 0], trainInds[:, :, 1], trainInds[:, :, 2]), axis = 1).T

    # Return an integer array
    return trainInds.astype(int), minTrain

# Formatting raw files
def lustigDataFormatting(mat_file, lfp_file, dec=50):
    """
    Input:
    mat_file = metadata file of type .mat
    lfp_file = raw lfp data file of type .lfp
    dec = decimation value

    Output:
    X = formatted lfp data
    lapID = formatted metadata
    sp = 2D histogram of spikes between time and channels

    Description:
    This function takes raw metadata and lfp files and returns a formatted
    metadata and lfp variables to undergo further formatting.
    """

    # Extract the needed data from the LFP.mat file.
    mat = io.loadmat(mat_file, variable_names = ['Track', 'xml', 'Spike', 'Clu'])
    Track = mat['Track']
    xml = mat['xml']
    Spike = mat['Spike']
    Clu = mat['Clu']

    # Extract the information from the bit .lfp data file and reshape to the appropriate matrix.
    a = np.memmap(lfp_file, 'int16')
    a = a.reshape((-1, xml['nChannels'][0][0][0][0]))

    # Bin both dimensions using histogram2d.
    sp, xedges, yedges = np.histogram2d(np.squeeze(Spike['res'][0][0]/dec), np.squeeze(Spike['totclu'][0][0]) - 1, bins = (np.arange(np.ceil(max(Spike['res'][0][0])/dec) + 1), np.arange(max(Spike['totclu'][0][0]) + 1)) )
    sp = np.uint8(sp)

    # Clip the dimensions to ensure the appropriate number of electrodes. Use multiple [0]'s to access the desired data.
    if np.size(a, 1) > 256:
        a = a[:, :255]
        sp[:, Clu['shank'][0][0][0] > 32] = []
    else:
        a = a[:, :192]
        sp[:, Clu['shank'][0][0][0] > 24] = []

    # Create the data matrix on which preprocessing will take place.
    X = np.zeros((np.size(a, 1), math.ceil(np.size(a, 0)/dec)))
    for j in range(np.size(a, 1)):
        doubleArray = np.asarray(a[:, j], dtype = np.float64, order ='C')
        X[j,:] = decimate(doubleArray, dec)

    # Construct lapID metadata using compAnly
    lapID = compAnly(Track, 1250/dec)
    return X, lapID, sp

# Preprocessing, drawing from brianPyNet.m
def brianPyNet(data, width, dec = 8, numCrossVals = 5, numSessions = 1):
    """
    Input:
    data = a dictionary containing X, lapID, and sp
    dec = decimation value
    numCrossVals = number of cross validations to be generated
    numSessions = number of sessions of available data

    Output:
    save file = generates unique save files for each session and running/stillness
    combination. Format: .npy

    save file contains:
    X = input lfp data
    lapID = output information
    trainInds = indices for cross validations, used for the train-test split
    sp = 2D histogram of spikes between time and channels

    Description:
    This function constructs a complete preprocessed input (X) and output (lapID) for
    FieldNet, and constructs indices for the train-test split (trainInds). This
    function also splits the data into the running and stillness datasets used in
    the manuscript. Files are saved as dictionaries in .npy format.
    """

    import numpy as np
    import math
    import copy

    # Testing parameters
    # gpSizes = np.zeros((numCrossVals, 2))

    # Construct multiple cross validations
    for i in range(numSessions):

        # Separate running and stillness
        LAPID = data['lapID']
        for j in range(1, 3): #4?

            # Construct output lapID and lfp data for each output
            inds = LAPID[:, 3] == j
            if j == 2:
                inds = np.logical_and(inds, LAPID[:, 2] == 1) # use & alias
            lapID = LAPID[inds, 0:2]
            lapID = np.insert(lapID, 1, 0, axis = 1)
            lapID = np.append(lapID, np.where(inds)[0][:, np.newaxis], 1).astype(int)
            
            Xfull = data['X']
            X = Xfull[inds, :]


            # Construct array of training indices
            trs = np.unique(lapID[:, 0])
            trs = trs[np.random.permutation(len(trs))]
            trainInds = np.array([])

            # Loop through 100 different fold assignments to find the largest one
            for k in range(100):
                # Assign a cross validation fold to each represented trial
                for l in range(len(trs)):
                    trialInds = lapID[:, 0] == trs[l]
                    if sum(trialInds) != 0:
                        lapID[trialInds, 1] = math.ceil(np.random.rand(1)[0] * numCrossVals)
                temp, _ = trainTestSplit(lapID, 20000)
                if temp.shape[0] > trainInds.shape[0]:
                    trainInds = copy.deepcopy(temp)
                    lapBest = copy.deepcopy(lapID)
                    # gpSizes[l, j - 1] = trainInds.shape[0] ## An internal check for sizes between loops
            lapID = lapBest

            # Save a .npy file for each session and running/stillness combo
            data_output = {'X': X, 'lapID': lapID, 'trainInds': trainInds, 'sp': data['sp']}
            np.save(f"pyDat{width}{i}{j-1}.npy", data_output)

In [8]:
#Filter functions!

def filtHigh(sig, fs, fH, order = 2):
    #Highpass filter for full broadband data

    # Construct the numerator and denominator for the filtfilt function
    b, a = signal.sos2tf(signal.butter(order, fH, btype='highpass', fs = fs, analog=False, output = 'sos'))

    # Apply the highpass filter
    sig = signal.filtfilt(b, a, sig.T).T

    return sig

def filter(matrix, range, btype='bp', fs = 1250/8, order = 3):
    #Bandpass filter for specific broadbands
    filter = signal.butter(order, range, btype, output = 'sos', fs = fs)
    filt_X = signal.sosfiltfilt(filter, matrix, axis= 1)
    return filt_X 

In [9]:
# Specify target files for data drudging/formatting
mat_file = "rec01_BehavElectrDataLFP.mat" # File name: "rec01_BehavElectrDataLFP.mat"
lfp_file = "rec01.lfp" # File name: "rec01.lfp"

# Generate dictionary containing X, lapID, and sp.
X, lapID, sp = lustigDataFormatting(mat_file, lfp_file, dec = 8)

#Filter data into a given bandwidth
X_full = filtHigh(X.T, 1250/8, 2)
theta_X = filter(X_full, range= [6,12], btype= 'bp',order= 2) 
harm_X = filter(X_full, range= [14,20], btype= 'bp',order= 2) 

H_X = signal.hilbert(X_full, axis= 0)
Htheta_X = signal.hilbert(theta_X, axis= 0)
Hharm_X = signal.hilbert(harm_X, axis= 0)
Hdouble_X = np.abs(Htheta_X)*np.exp(1j*np.angle(Htheta_X)*2)

data = {'X': H_X, 'lapID': lapID, 'sp': sp}
theta_data = {'X': Htheta_X, 'lapID': lapID, 'sp': sp}
harm_data = {'X': Hharm_X, 'lapID': lapID, 'sp': sp}
double_data = {'X': Hdouble_X, 'lapID': lapID, 'sp': sp}

# Data separation and formatted file creation
data_output = brianPyNet(data, width = "Full", dec = 8)
theta_output = brianPyNet(theta_data, width = "Theta", dec = 8)
harm_output = brianPyNet(harm_data, width = "Harm", dec = 8)
double_output = brianPyNet(double_data, width = "Double", dec = 8)